# **Optical camera communications image processing for receiver**

> Python script for the image processing part of the receiver of an optical camera comuunication system.

> The objective is to demodulate a message, using on-off keying modulation/demodulation.

> The python script will import an image and try to demodulate it by image processing methods.

> The python script will be executed on a Raspberry pi 4 or a pc.

# **Necessary imports**

In [1]:
import numpy as np
import cv2
import sys
import cv2
import matplotlib.pyplot as plt
import time

# **The sub-sampling method**

In [4]:
# Logical variables for control

# List for the transmited message
message_list = []

# Start the video feed
cap = cv2.VideoCapture(0) # the right value usually is 0 or 1

if (cap.isOpened() == False):
    print("Error opening video camera")

y_min = 80
y_max = 400
x_min = 360
x_max = 665

# Read until video is completed
while(cap.isOpened()):
     
# Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        
        # Grayscale image and thresholding
        grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        (thresh, bnwFrame) = cv2.threshold(grayframe, 50, 255, cv2.THRESH_BINARY)

        # # Just for testing
        # counting the number of pixels
        # number_of_white_pix = np.sum(bnwFrame == 255)
        # number_of_black_pix = np.sum(bnwFrame == 0)
        # print('Number of white pixels:', number_of_white_pix)
        # print('Number of black pixels:', number_of_black_pix)
        # print(frame.shape)

        # detection procedure and/or frame cropping

        # Cropping: extract useful part of the frame (manually at first)
        useful_frame = bnwFrame[y_min:y_max, x_min:x_max]
        # # for testing
        # cv2.imshow('frame', useful_frame)
        # print(bnwFrame.shape)
        # print(useful_frame.shape)

        # # find the new image resolution
        row_num = y_max - y_min # height
        col_num = x_max - x_min # length
        # start storing the bits to the message list

        # for each row with step == thickness
        thickness = 70 # pixels (change manually)
        threshold = 50 # pixels (change manually)
        for y in range(int(thickness/2), int(row_num - thickness/2), thickness):
            row = useful_frame[y, 0:col_num-1] # get the row
            holder = np.sum(row == 255) # add to holder the sum of white pixels
            if holder > threshold:
                message_list.append(1)
            else:
                message_list.append(0)
            
            # plt.plot(new)
            # plt.ylabel('Number of white pixels')
            # plt.xlabel('Row number')
            # plt.show()
            # print(useful_frame.shape) # just testing

        # just testing
        cv2.imshow('frame', useful_frame)
         
    # Press Q on keyboard to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Break the loop
    else:
        break

# When everything done, release
# the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

print(message_list)

# convert binary to jpg, png, mp3, raw, text...
# show the received message

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

# **Compare lists test**

In [8]:
start_sequence = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
temp = []
for i in range(29):
    temp.append(1)
temp.append(0)
N = 5

if start_sequence == temp:
    print("equal")
elif start_sequence != temp:
    print("not equal")
a = start_sequence[-N:]
print(a)
print(len(start_sequence))

equal
[1, 1, 1, 1, 0]
30


# **Sub-sampling test**

In [2]:
import cv2

# Open the video file
cap = cv2.VideoCapture(0)

y_min = 80
y_max = 400
x_min = 360
x_max = 665
frame_width = x_max - x_min
frame_height = y_max - y_min
thickness = 70 # pixels (change manually)

while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()

    if ret:
        grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        (thresh, bnwFrame) = cv2.threshold(grayframe, 50, 255, cv2.THRESH_BINARY) # change the threshold to what is needed
        useful_frame = bnwFrame[y_min:y_max, x_min:x_max]
        # for each row with step == thicknesσ
        for y in range(0, frame_height, thickness):
            cv2.line(useful_frame, (0, y), (frame.shape[0], y), (0, 0, 255), 2)

        # Display the frame
        cv2.imshow('frame', useful_frame)
        
        # Exit if 'q' is pressed
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

# Release the resources
cap.release()
cv2.destroyAllWindows()



# **Change detection method**

In [16]:
# Open the video file
cap = cv2.VideoCapture(0)

y_min = 60
y_max = 381
x_min = 361
x_max = 682
frame_width = x_max - x_min
frame_height = y_max - y_min
message_list = []
thickness = 45 # minimum thickness of the bit row set
threshold = 0
no = 0
while cap.isOpened():

    # Read a frame from the video
    print("start to read")
    print(round(time.time() * 1000))
    ret, frame = cap.read()
    print("end reading")
    print(round(time.time() * 1000))

    if ret:
        grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        (thresh, bnwFrame) = cv2.threshold(grayframe, 50, 255, cv2.THRESH_BINARY) # change the threshold to what is needed
        useful_frame = bnwFrame[y_min:y_max, x_min:x_max]

        # ***Change detection process***
        holder = []

        # store the profil of the frame
        for y in range(0, frame_height):
            row = useful_frame[y, 0:frame_width-1] # get the row
            if np.sum(row == 255) > threshold:
                holder.append(1)
            else:
                holder.append(0)

        # # translate into data
        # edge_thickness = 40
        # if counter > edge_thickness:
        #     if holder[0] == 0:
        #         message_list.append(0)
        #     else:
        #         message_list.append(1)
        #     counter = 0
        # for i in range(1, len(holder)):
        #     if holder[i] != holder[i - 1]:
        #         message_list.append(holder[i])
        #         counter = 0
        #     else:
        #         counter += 1
        #     if counter >= thickness:
        #         message_list.append(holder[i])
        #         counter = 0

        # counter1 = 0
        # counter0 = 1
        # for i in range(0, len(holder)):
        #     if holder[i] == 1:
        #         counter1 += 1
        #         if counter1 >= thickness:
        #             message_list.append(holder[i])
        #             counter1 = 0
        #     elif holder[i] == 0:
        #         counter0 += 1
        #         if counter0 >= thickness:
        #             message_list.append(holder[i])
        #             counter0 = 0


        current_num = holder[0]
        current_count = 1
        groups = []

        for num in holder[1:]:
            if num == current_num:
                current_count += 1
            else:
                groups.append((current_num, current_count))
                current_num = num
                current_count = 1

        groups.append((current_num, current_count))

        message_list.append(groups[0][0])
        for i in range(1, len(groups)):
            if groups[i][1] >= thickness:
                message_list.append(groups[i][0])
        
        # print(groups)


        # Display the frame
        cv2.imshow('frame', useful_frame)
        # cv2.imwrite('frame'+str(i)+'.jpg',useful_frame)
        # no+=1
        # print(message_list)
        
        # Exit if 'q' is pressed
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

print(message_list)

# Release the resources
cap.release()
cv2.destroyAllWindows()

start to read
1683641505469
end reading
1683641506788
start to read
1683641506840
end reading
1683641506847
start to read
1683641506886
end reading
1683641506939
start to read
1683641506970
end reading
1683641506977
start to read
1683641507014
end reading
1683641507021
start to read
1683641507068
end reading
1683641507075
start to read
1683641507115
end reading
1683641507120
start to read
1683641507162
end reading
1683641507166
start to read
1683641507209
end reading
1683641507214
start to read
1683641507256
end reading
1683641507261
start to read
1683641507303
end reading
1683641507307
start to read
1683641507337
end reading
1683641507343
start to read
1683641507385
end reading
1683641507441
start to read
1683641507486
end reading
1683641507492
start to read
1683641507530
end reading
1683641507535
start to read
1683641507577
end reading
1683641507579
start to read
1683641507608
end reading
1683641507611
start to read
1683641507655
end reading
1683641507659
start to read
1683641507702


# **Performance testing**

In [ ]:
errors = []
message = []
message_list = []
if message == message_list:
    print("Message received successfully")
else:
    for i in range(len(message_list)):
        if message_list[i] != message[i]:
            errors.append(i)
print("The errors are at the positions:")
print(errors)

print("Total errors are:")
print(len(errors))

print("Error percentage:")
print(len(errors)/len(message))

# **Post processing testing**

In [59]:
import cv2
import numpy as np

cap = cv2.VideoCapture('MVI_7057.MP4')
y_min = 320
y_max = 840
x_min = 706
x_max = 1247
frame_width = x_max - x_min
frame_height = y_max - y_min
threshold = 0
groups = []

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
 
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
 
# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
 
    grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    (thresh, bnwFrame) = cv2.threshold(grayframe, 70, 255, cv2.THRESH_BINARY) # change the threshold to what is needed
    useful_frame = bnwFrame[y_min:y_max, x_min:x_max]

    # ***Change detection process***
    holder = []

    # store the profil of the frame
    for y in range(0, frame_height):
        row = useful_frame[y, 0:frame_width-1] # get the row
        if np.sum(row == 255) > threshold:
            holder.append(1)
        else:
            holder.append(0)


    current_num = holder[0]
    current_count = 1

    for num in holder[1:]:
        if num == current_num:
            current_count += 1
        else:
            groups.append((current_num, current_count))
            current_num = num
            current_count = 1

    groups.append((current_num, current_count))

    # Display the resulting frame
    cv2.imshow('Frame',useful_frame)
 
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
 
  # Break the loop
  else: 
    break
 
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

print(groups)

[(1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 519), (0, 1), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 520), (1, 519), (0, 1), (1, 520), (1, 520), (1, 520), (1, 520), (1,

In [60]:
received = []
counter = 0
for element in groups:
    if element[1] > 10 and element[1] < 500:
        print(element)
        counter += 1
        received.append(element[0])
print("The counter is:")
print(counter)

(1, 91)
(0, 426)
(1, 230)
(0, 290)
(1, 347)
(0, 173)
(0, 18)
(1, 447)
(0, 52)
(0, 132)
(1, 388)
(0, 250)
(1, 268)
(0, 367)
(1, 150)
(1, 11)
(0, 476)
(1, 33)
(1, 132)
(0, 388)
(1, 249)
(0, 271)
(1, 363)
(0, 157)
(0, 34)
(1, 447)
(0, 37)
(0, 148)
(1, 370)
(0, 268)
(1, 252)
(0, 382)
(1, 135)
(1, 26)
(0, 473)
(1, 18)
(1, 144)
(0, 376)
(1, 261)
(0, 256)
(1, 378)
(0, 142)
(0, 49)
(1, 446)
(0, 25)
(0, 163)
(1, 355)
(0, 287)
(1, 233)
(0, 404)
(1, 116)
(1, 46)
(0, 472)
(1, 166)
(0, 354)
(1, 283)
(0, 237)
(1, 400)
(0, 120)
(0, 68)
(1, 446)
(0, 182)
(1, 336)
(0, 299)
(1, 221)
(0, 415)
(1, 105)
(1, 57)
(0, 460)
(1, 173)
(0, 347)
(1, 284)
(0, 233)
(1, 400)
(0, 120)
(0, 68)
(1, 446)
(0, 182)
(1, 335)
(0, 306)
(1, 214)
(0, 420)
(1, 98)
(1, 68)
(0, 452)
(1, 182)
(0, 336)
(1, 298)
(0, 222)
(1, 416)
(0, 101)
(0, 83)
(1, 437)
(0, 201)
(1, 317)
(0, 318)
(1, 202)
(0, 435)
(1, 85)
(1, 80)
(0, 437)
(1, 197)
(0, 320)
(1, 314)
(0, 203)
(1, 431)
(0, 89)
(0, 98)
(1, 422)
(0, 215)
(1, 305)
(0, 332)
(1, 188)
(0, 4

In [27]:
message = np.zeros(500)
for i in range(message.size):
  if (i % 2) == 0:
    message[i] = 1

In [61]:
errors = []
message_list = []
if message == received:
    print("Message received successfully")
else:
    for i in range(1, len(received)):
        if received[i] == received[i - 1]:
            errors.append(i)

print("Total errors are:")
print(len(errors))
print(errors)

print("Error percentage:")
print(len(errors)/len(message))

Total errors are:
92
[6, 9, 15, 18, 24, 27, 33, 36, 42, 45, 51, 59, 67, 75, 83, 91, 99, 107, 115, 123, 131, 139, 147, 153, 156, 164, 172, 180, 188, 196, 202, 205, 213, 219, 222, 230, 236, 239, 245, 248, 254, 257, 263, 266, 272, 275, 281, 284, 290, 293, 299, 302, 308, 311, 317, 320, 326, 329, 335, 338, 344, 347, 353, 356, 362, 365, 371, 374, 380, 383, 391, 397, 400, 408, 414, 417, 425, 431, 434, 442, 450, 458, 466, 474, 482, 490, 498, 506, 514, 522, 530, 538]
Error percentage:
0.184


C:\Users\Christos\AppData\Local\Temp\ipykernel_6912\2926533898.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if message == received:
